<a href="https://colab.research.google.com/github/chirag2saraiya/Session-16_DenseNet_TFRecord/blob/master/DNST_CIFAR10_AUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://keras.io/
!pip install -q keras
import keras
!rm -rf "/content/DeepLearning"
!git clone https://github.com/chirag2saraiya/DeepLearning.git

Using TensorFlow backend.


Cloning into 'DeepLearning'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 121 (delta 59), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (121/121), 66.80 KiB | 3.93 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from DeepLearning import tfrecords
from DeepLearning import earlystop_callback

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
tfrecords.convert('/content/cifar10')

Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Generating /content/cifar10/train.tfrecords


Generating /content/cifar10/test.tfrecords
Done!


In [6]:
x_train,y_train = tfrecords.Cifar10DataSet("/content/cifar10").make_batch(batch_size)
print(x_train)
print(y_train)
x_test,y_test = tfrecords.Cifar10DataSet("/content/cifar10","test",False).make_batch(batch_size)
print(x_test)
print(y_test)





Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Tensor("IteratorGetNext:0", shape=(?, 32, 32, 3), dtype=float32)
Tensor("IteratorGetNext:1", shape=(?, 10), dtype=float32)
Tensor("IteratorGetNext_1:0", shape=(?, 32, 32, 3), dtype=float32)
Tensor("IteratorGetNext_1:1", shape=(?, 10), dtype=float32)


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [10]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(tensor=x_train)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [12]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'],
              target_tensors=[y_train])



In [13]:

filepath= "/content/best_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint,earlystop_callback.TerminateOnBaseline(monitor='val_acc', baseline=0.85)]


model.fit(epochs=150,steps_per_epoch=100,verbose=1,validation_data=(x_test,y_test),validation_steps = 100,callbacks = callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/150
100/100 [==============================] - 130s 1s/step - loss: 2.0094 - acc: 0.2471 - val_loss: 1.9055 - val_acc: 0.2994

Epoch 00001: val_acc improved from -inf to 0.29938, saving model to /content/best_model.hdf5
Epoch 2/150
100/100 [==============================] - 106s 1s/step - loss: 1.7041 - acc: 0.3609 - val_loss: 2.4564 - val_acc: 0.2938

Epoch 00002: val_acc did not improve from 0.29938
Epoch 3/150
100/100 [==============================] - 106s 1s/step - loss: 1.5666 - acc: 0.4147 - val_loss: 1.8796 - val_acc: 0.3957

Epoch 00003: val_acc improved from 0.29938 to 0.39574, saving model to /content/best_model.hdf5
Epoch 4/150
100/100 [==============================] - 106s 1s/step - loss: 1.4620 - acc: 0.4622 - val_loss: 1.8952 - val_acc: 0.3991

Epoch 00004: val_acc improved from 0.39574 to 0.39906, saving model to /content/best_model.hdf5
Epoch 5/150
100/100 [==========